In [1]:
from langchain_community.document_loaders.generic import GenericLoader
from langchain_community.document_loaders.parsers import LanguageParser
from langchain_text_splitters import Language, RecursiveCharacterTextSplitter
from langchain_chroma import Chroma 
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains.question_answering import load_qa_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

import os
from git import Repo

In [2]:
repo_path = "./test_repo"

In [ ]:
repo = Repo.clone_from("https://github.com/langchain-ai/langchain", to_path= repo_path)

In [3]:
loader = GenericLoader.from_filesystem(
    repo_path + "/libs/core/langchain_core/",
    glob = "**/*",
    suffixes = [".py"],
    exclude = ["**/non-utf-8-encoding.py"],
    parser = LanguageParser(language=Language.PYTHON, parser_threshold=500)
)

documents = loader.load()
len(documents)

569

In [4]:
python_splitter = RecursiveCharacterTextSplitter.from_language(
    language= Language.PYTHON, chunk_size = 2000, chunk_overlap = 200, 
)

texts =  python_splitter.split_documents(documents)
len(texts)

1591

In [ ]:
os.environ['OPENAI_API_KEY'] = token

In [13]:
db = Chroma.from_documents(texts, OpenAIEmbeddings(disallowed_special=()))

retriever = db.as_retriever(
    search_type = 'mmr', 
    search_kwargs = {"k" : 8}
)

In [14]:
llm = ChatOpenAI(model="gpt-3.5-turbo", max_tokens = 200)

In [16]:
prompt = ChatPromptTemplate.from_messages(
    [(
        "system", #instrução que fica salva para ser usada pelo modelo
        "Você é um revisor de código experiente. Forneça informações detalhadas sobre a revisão do código e sugestões de melhorias baseado no contexto fornecido abaixo: \n\n {context}"
    ), 
        ("user", #mensagem do usuário
         "{input}"
        )
    ]
)

document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [19]:
response = retrieval_chain.invoke({"input": "Você pode revisar e sugerir melhorias para o código de RunnableBinding"})

In [21]:
print(response['answer'])

1. **Tipagem de Parâmetros**: 
   - Garanta que a tipagem dos parâmetros e do retorno dos métodos estão corretos e coerentes com o que é esperado. Por exemplo, `Any` pode ser muito genérico em alguns casos, tente ser mais específico quando possível.

2. **Documentação**:
   - Certifique-se de que todos os métodos e classes tenham uma documentação adequada, explicando o seu propósito, os parâmetros esperados, o que retorna, e exemplos de uso quando necessário.

3. **Clareza e Legibilidade**:
   - Quebre partes do código muito extensas em métodos menores e mais específicos para facilitar a leitura e manutenção.
   - Utilize nomes de variáveis mais descritivos para melhorar a compreensão do código.

4. **Uso de Decoradores**:
   -
